<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/analysis/poisson%20gamma%20distribution%20for%20crash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 단기 폭락 (1~3개월): 경제 지표 악화, 정치적 불안, 자연재해 등 단기적인 사건으로 인해 발생하며, 10%~20% 정도의 폭락이 일반적입니다.
# 중기 폭락 (6개월~1년): 경기 침체, 금융 위기 등 중기적인 경제 악화로 인해 발생하며, 20%~30% 정도의 폭락이 일반적입니다.
# 장기 폭락 (1년 이상): 대공황과 같은 심각한 경제 위기로 인해 발생하며, 50% 이상의 폭락도 가능합니다.


# 1개월 이상 3개월 이내 10%~20% 폭락
# 6개월 이상 1년 이내 20~30% 폭락
# 1년 이상 50% 폭락

In [2]:
import pandas as pd

def get_df(url):
  df = pd.read_csv(url)
  df['Date'] = pd.to_datetime(df['Date'])
  df.index = df['Date']
  df.drop(columns=['Date'], inplace=True)
  return df

In [3]:
df = get_df('https://stooq.com/q/d/l/?s=^SPX&i=d')

In [4]:
df

,Open,High,Low,Close,Volume
Date,,,,,
1789-05-01,0.51,0.51,0.51,0.51,0.000000e+00
1789-06-01,0.51,0.51,0.51,0.51,0.000000e+00
1789-07-01,0.50,0.50,0.50,0.50,0.000000e+00
1789-08-01,0.50,0.51,0.50,0.51,0.000000e+00
1789-09-01,0.51,0.51,0.50,0.51,0.000000e+00
...,...,...,...,...,...
2024-05-20,5305.35,5325.32,5302.40,5308.13,1.850086e+09
2024-05-21,5298.69,5324.32,5297.87,5321.41,1.989596e+09
2024-05-22,5319.28,5323.18,5286.01,5307.01,2.094854e+09


In [5]:
import pandas as pd

def find_declines(df, min_period, max_period, decline_threshold):
    declines = []
    decline_id = 1

    for start_date in df.index:
        for period in range(min_period, max_period + 1):
            end_date = start_date + pd.DateOffset(months=period)
            if end_date not in df.index:
                continue

            start_price = df.loc[start_date, 'Close']
            end_price = df.loc[end_date, 'Close']
            decline_percentage = ((end_price - start_price) / start_price) * 100

            # 현재 폭락 사건과 겹치는 이전 사건이 있는지 확인
            overlapping = False
            for prev_decline in declines:
                prev_start_date = pd.Timestamp(prev_decline['start_date'])
                prev_end_date = pd.Timestamp(prev_decline['end_date'])
                if (prev_start_date <= start_date <= prev_end_date) or \
                   (prev_start_date <= end_date <= prev_end_date):
                    overlapping = True
                    break

            # 겹치는 사건이 없을 때만 추가
            if not overlapping and decline_percentage <= decline_threshold:
                declines.append({
                    'id': decline_id,
                    'period': period_label(min_period, max_period),
                    'start_date': start_date.strftime('%Y-%m-%d'),
                    'end_date': end_date.strftime('%Y-%m-%d'),
                    'decline_percentage': round(decline_percentage, 2)
                })
                decline_id += 1

    return declines

def period_label(min_period, max_period):
    if max_period <= 3:
        return '단기'
    elif max_period <= 12:
        return '중기'
    else:
        return '장기'

# 단기 폭락: 1개월 이상 3개월 이내 10%~20% 폭락
short_term_declines = find_declines(df, 1, 3, -10)

# 중기 폭락: 6개월 이상 1년 이내 20~30% 폭락
mid_term_declines = find_declines(df, 6, 12, -20)

# 장기 폭락: 1년 이상 50% 폭락
long_term_declines = find_declines(df, 12, 24, -50)

# 결과 병합
all_declines = short_term_declines + mid_term_declines + long_term_declines

In [15]:
import plotly.graph_objects as go

# 주가 데이터를 로그 스케일로 변환하여 플롯
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='주가', line=dict(color='blue')))
fig.update_layout(yaxis_type='log')  # y축을 로그 스케일로 설정

# 폭락 사건의 기간을 막대 그래프로 표시
for decline in short_term_declines:
    start_date = decline['start_date']
    end_date = decline['end_date']
    fig.add_shape(type='rect',
                  x0=start_date, y0=min(df['Close']), x1=end_date, y1=max(df['Close']),
                  line=dict(color='red', width=1),
                  fillcolor='red', opacity=0.3,
                  layer='below')

fig.update_layout(title='단기 폭락',
                  xaxis=dict(title='날짜'),
                  yaxis=dict(title='주가 (로그 스케일)'))
fig.show()


In [16]:
import plotly.graph_objects as go

# 주가 데이터를 로그 스케일로 변환하여 플롯
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='주가', line=dict(color='blue')))
fig.update_layout(yaxis_type='log')  # y축을 로그 스케일로 설정

# 폭락 사건의 기간을 막대 그래프로 표시
for decline in mid_term_declines:
    start_date = decline['start_date']
    end_date = decline['end_date']
    fig.add_shape(type='rect',
                  x0=start_date, y0=min(df['Close']), x1=end_date, y1=max(df['Close']),
                  line=dict(color='red', width=1),
                  fillcolor='red', opacity=0.3,
                  layer='below')

fig.update_layout(title='중기 폭락',
                  xaxis=dict(title='날짜'),
                  yaxis=dict(title='주가 (로그 스케일)'))
fig.show()


In [17]:
import plotly.graph_objects as go

# 주가 데이터를 로그 스케일로 변환하여 플롯
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='주가', line=dict(color='blue')))
fig.update_layout(yaxis_type='log')  # y축을 로그 스케일로 설정

# 폭락 사건의 기간을 막대 그래프로 표시
for decline in long_term_declines:
    start_date = decline['start_date']
    end_date = decline['end_date']
    fig.add_shape(type='rect',
                  x0=start_date, y0=min(df['Close']), x1=end_date, y1=max(df['Close']),
                  line=dict(color='red', width=1),
                  fillcolor='red', opacity=0.3,
                  layer='below')

fig.update_layout(title='장기 폭락',
                  xaxis=dict(title='날짜'),
                  yaxis=dict(title='주가 (로그 스케일)'))
fig.show()


In [31]:
import numpy as np
from scipy.stats import gamma
import pandas as pd

def next_decline_probability(months, df, decline_periods):
    # 폭락 간의 시간 간격 계산
    decline_intervals = [(pd.to_datetime(decline_periods[i+1]['start_date']) - pd.to_datetime(decline_periods[i]['start_date'])).days
                         for i in range(len(decline_periods)-1)]

    # 감마 분포 모수 추정
    alpha, loc, beta = gamma.fit(decline_intervals)

    # 다음 주어진 개월 수 이내에 폭락이 발생할 확률 계산
    next_probability = gamma.cdf(months * 30, a=alpha, loc=loc, scale=beta)

    return next_probability



In [47]:
next_decline_probability(, df, short_term_declines)

0.9994691200970257

In [30]:
next_decline_probability(12, mid_term_declines)

47


0.17000727077279762

In [13]:
next_decline_probability(12, long_term_declines)


0.0

In [52]:
import numpy as np
from scipy.stats import gamma
import pandas as pd
import plotly.graph_objects as go

# 1개월부터 24개월까지의 폭락 발생 확률 계산
months_range = range(1, 25)
probabilities = [next_decline_probability(months, df, short_term_declines) for months in months_range]

# 그래프 그리기
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(months_range), y=probabilities, mode='lines+markers', name='폭락 발생 확률'))

fig.update_layout(
    title='향후 개월 수에 따른 폭락 발생 확률',
    xaxis_title='개월 수',
    yaxis_title='폭락 발생 확률',
    xaxis=dict(tickmode='linear'),
    yaxis=dict(tickformat=".2%")
)

fig.show()

In [14]:
from scipy.stats import poisson

# 총 기간 계산 (1986년부터 2024년까지)
total_years = 2024 - 1986 + 1  # 2024년 포함

# 연평균 중기 폭락 횟수
average_annual_declines = 7 / total_years

# 특정 기간 (예: 3년) 동안 중기 폭락이 발생할 확률 계산
def poisson_probability(years, average_annual_rate):
    # 평균 이벤트 발생 횟수 (λ)
    lambda_ = average_annual_rate * years
    # 특정 횟수 이상 이벤트가 발생할 확률 계산 (예: 1번 이상)
    probability = 1 - poisson.cdf(0, lambda_)
    return probability

# 3년 동안 중기 폭락이 한 번 이상 발생할 확률
prob_3_years = poisson_probability(3, average_annual_declines)
print(f"다음 3년 이내에 중기 폭락이 한 번 이상 발생할 확률: {prob_3_years:.4f}")

# 6개월 동안 중기 폭락이 한 번 이상 발생할 확률
prob_6_months = poisson_probability(0.5, average_annual_declines)
print(f"다음 6개월 이내에 중기 폭락이 한 번 이상 발생할 확률: {prob_6_months:.4f}")


다음 3년 이내에 중기 폭락이 한 번 이상 발생할 확률: 0.4164
다음 6개월 이내에 중기 폭락이 한 번 이상 발생할 확률: 0.0858
